# 0. Instalar e importar bibliotecas 📚

In [1]:
# !pip install requests
# !pip install requests_oauthlib
# !pip install oauthlib
# !pip install pandas

In [2]:
import requests
from requests_oauthlib import OAuth2Session
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
from oauthlib.oauth2 import BackendApplicationClient
import json

In [3]:
from datetime import datetime as dt
import pandas as pd

In [4]:
from rjson import RenderJSON #funcionalidade para colapsar as saídas em json

# 1. Obter token de acesso 🔑

Usar o arquivo json gerado no notebook credentials com client_id e client_secret.

In [5]:
keys = json.loads(open('keys.json', 'r').read())

Ou preencher manualmente os dados das credenciais de acesso

In [6]:
# keys = {
#     "client_id": "PREENCHER COM CLIENT ID",
#     "client_secret": "PREENCHER COM CLIENT SECRET",
# }

In [7]:
client_id = keys['client_id']
client_secret = keys['client_secret']
scope= "accountability.statements accountability.request"

In [8]:
def get_token(client_id, client_secret):
    auth = HTTPBasicAuth(client_id, client_secret)
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client) 
    oauth = OAuth2Session(client=client, scope=scope)
    return oauth.fetch_token(token_url='https://oauth.hm.bb.com.br/oauth/token', auth=auth)
token = get_token(keys['client_id'], keys['client_secret'])

In [9]:
print('validade do token: ', dt.strftime(dt.fromtimestamp(token['expires_at']), '%d/%m/%y %H:%M:%S'))

validade do token:  01/08/23 12:06:39


# 2. Requisições 📞

Criar cabeçalho com a chave da aplicação de homologação e token de acesso.

In [10]:
def set_headers():
    gw_dev_app_key = keys['gw_dev_app_key']
    headers = {
            'accept': 'application/json',
            'Authorization': 'Bearer '+token['access_token'],
            'X-Application-Key' : gw_dev_app_key
            }
    return headers

In [15]:
path = 'https://api.hm.bb.com.br/accountability/v3'

## API 📕

In [16]:
token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()
r = requests.get(f"{path}/swagger", headers=headers)
# r.json()

In [17]:
RenderJSON(r.json())

2. Índice
    * [/agencias-proximas](##Agencias) ✅
    * [Programas](##Programas)
        * [/programas-governo/{numeroProgramaGoverno}/categorias](###Progamas-categorias) ✅
        * [/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/lancamentos-atualizados](###Progamas-lancamentos) 🟥
        * [/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/sublancamentos-atualizados](###Progamas-sublancamentos) 🟥
    * [Extratos](##Extratos)
        * [/statements/{branchCode}-{accountNumber}](###Extratos-transacoes) ✅
        * [/statements/{branchCode}-{accountNumber}/control-agencies]() ✅
        * [/statements/{branchCode}-{accountNumber}/debits/{id}/subtransactions](###Extratos-subtransacoes) ✅
        * [/statements/{branchCode}-{accountNumber}/debits/{id}/control-agencies/subtransactions]() 🟥
    * [Aplicações](##Aplicacoes)
        * [/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}](###Aplicacoes-fundos) ✅
        * [/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}/control-agencies]() ❎
        * [/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}](###Aplicacoes-poupanca) ✅
        * [/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}/orgao-controle]() ❎
    * [Despesas](##Despesas)
        * [/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/documents/{documentId}](###Despesas-transacao-documentos) ❎
            * [/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/subTransactions/{subTransactionId}/documents/{documentId](###Despesas-subtransacao-documentos) ❎
    * [Saldos](##Saldos)
        * [/saldos/{agencia}-{contaCorrente}/conta-corrente]() ✅
        * [/saldos/{agencia}-{contaCorrente}/aplicacoes-financeiras]() ❎
    * [Orgãos](##Orgaos)
        * [/orgaos-repasse/{agencia}-{contaCorrente}//lancamentos-debito]() ❎
        * [/orgaos-repasse/lancamentos-credito/{ordemBancaria}-{item}/categorias-despesa]() ⬜
        * [/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-credito]() 🟥
        * [/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-credito/{sequencialLancamento}-{sequencialIdentificacao]() ⬜

## Agencias

In [18]:
cnpj = '11435633000149'
cep = '50050910'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/agencias-proximas?cnpj={cnpj}&cep={cep}", headers=headers)
RenderJSON(r.json())

## Programas

### Progamas-categorias

In [19]:
numeroProgramaGoverno = "3"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/categorias", headers=headers)

RenderJSON(r.json())

### Progamas-lancamentos

In [20]:
numeroProgramaGoverno = "3"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'dataInicio': '2018-10-01',
    'dataFim': '2018-10-30',
    'pagina': 1
}

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/lancamentos-atualizados", headers=headers, params=body)

RenderJSON(r.json())

### Progamas-sublancamentos

In [21]:
numeroProgramaGoverno = "3"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'dataInicio': '2018-10-01',
    'dataFim': '2018-10-30',
    'pagina': 1
}

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/sublancamentos-atualizados", headers=headers, params=body)

RenderJSON(r.json())

## Extratos

### Extratos-transacoes

In [22]:
branchCode = "551"
accountNumber = "5420"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/statements/{branchCode}-{accountNumber}?startDate={data_inicio}&endDate={data_fim}", headers=headers)
RenderJSON(r.json())

In [23]:
branchCode = "551"
accountNumber = "5420"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/statements/{branchCode}-{accountNumber}/control-agencies?startDate={data_inicio}&endDate={data_fim}", headers=headers)
RenderJSON(r.json())

In [24]:
extrato = r.json()
pd.DataFrame(extrato['transactions'])

,id,bookingDate,orderIndex,valueDate,referenceNumber,value,accountBalance,descriptionCode,descriptionName,descriptionBatchNumber,...,pendingExpenseConciliation,attachedExpenseDocumentIndicator,expenseCategoryCode,expenseIdentificationStatus,subTransactionQuantity,bankOrderPurposeCode,bankOrderRuleCode,bankOrderPurposeDescription,expensesCategory,expensesDocuments
0,6,04/10/2018,1,04/10/2018,5205306000035,4182.92,0.0,132,Emissão de Ordem Bancária,13138,...,False,False,87,0,2,2,0,,"[{'code': 87, 'parentCode': 0, 'name': 'Vigilâ...",[]
1,8,04/10/2018,2,04/10/2018,5211705000015,2148.80,0.0,632,Ordem Bancária,14056,...,False,False,0,0,0,2,0,,[],[]
2,10,04/10/2018,3,04/10/2018,5217990000051,2310.80,NaN,632,Ordem Bancária,14056,...,False,False,0,0,0,2,0,,[],[]
3,11,04/10/2018,4,04/10/2018,70,6792.20,0.0,345,Aplicação em BB Fix,0,...,True,True,87,0,0,2,0,,"[{'code': 87, 'parentCode': 0, 'name': 'Vigilâ...","[27, 35, 181]"
4,7,11/10/2018,1,11/10/2018,101101,17829.19,0.0,393,TED Transf.Eletr.Disponivel,13105,...,True,True,87,0,0,2,0,,"[{'code': 87, 'parentCode': 0, 'name': 'Vigilâ...","[26, 32, 39, 74, 240, 241]"
5,9,11/10/2018,2,11/10/2018,70,17829.19,0.0,855,Resgate BB Fix,0,...,False,False,0,0,0,2,0,,[],[]


### Extratos-subtransacoes

In [25]:
branchCode = "551"
accountNumber = "5420"
id = 6

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/statements/{branchCode}-{accountNumber}/debits/{id}/subtransactions", headers=headers)
RenderJSON(r.json())

In [26]:
# branchCode = "551"
# accountNumber = "5420"
# id = 6

# token = get_token(keys['client_id'], keys['client_secret'])
# headers = set_headers()

# r = requests.get(f"{path}/statements/{branchCode}-{accountNumber}/debits/{id}/control-agencies/subtransactions", headers=headers)
# r.json()

In [27]:
subextrato = r.json()
pd.DataFrame(subextrato['subtransactions'])

,id,codeSubtransactionState,paymentStatus,paymentDate,value,beneficiaryBankIdentifierCode,beneficiaryBranchCode,beneficiaryAccountNumber,beneficiaryPersonType,beneficiaryDocumentId,beneficiaryName,attachedExpenseDocumentIndicator,expenseCategoryCode,subtransactionAccountabilityIndicator,subtransactionAccountabilityName,bankOrderPurposeCode,bankOrderRuleCode,bankOrderPurposeDescription,expensesCategory,expensesDocuments
0,1,6,3,03.01.2019,-3553.27,0,0,0,2,53604999000144,FERRAGENS DE PAIVA DE MACEDO,True,7,True,Pago,0,0,,"[{'code': 7, 'name': 'Hortalicas e derivados',...",[106]
1,2,6,3,03.01.2019,-629.65,1,1812,118815,1,99948941101,,True,7,True,Pago,0,0,,"[{'code': 7, 'name': 'Hortalicas e derivados',...",[]


## Despesas

### Despesas-transacao-documentos

In [28]:
branchCode = "551"
accountNumber = "5420"
transactionId = "7"
documentId = "26"
bookingDate = "2018-10-11"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/documents/{documentId}?bookingDate={bookingDate}", headers=headers)
RenderJSON(r.json())

In [29]:
token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

for lancamento in extrato['transactions']:
    print(lancamento['id'])
    branchCode = "551"
    accountNumber = "5420"
    transactionId = lancamento['id']
    bookingDate = dt.strftime(pd.to_datetime(lancamento['bookingDate']), "%Y-%m-%d")
    if lancamento['attachedExpenseDocumentIndicator']:
        # print(lancamento['expensesDocuments'])
        for doc in lancamento['expensesDocuments']:
            documentId = doc
            r = requests.get(f"{path}/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/documents/{documentId}?bookingDate={bookingDate}", headers=headers)
            print(r.json())

6
8
10
11
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
7
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
{'errors': [{'code': '3164477.1', 'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}
9


### Despesas-subtransacao-documentos

In [30]:
branchCode = "551"
accountNumber = "5420"
transactionId = "6"
subTransactionId = "1"
documentId = "106"
bookingDate = "2018-10-11"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/subTransactions/{subTransactionId}/documents/{documentId}?bookingDate={bookingDate}", headers=headers)
RenderJSON(r.json())

## Aplicacoes

### Aplicacoes-fundos

In [31]:
agencia = "551"
contaCorrente = "5420"
fundosInvestimentoId = '1'


token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}", headers=headers, params=body)
RenderJSON(r.json())

In [32]:
agencia = "551"
contaCorrente = "5420"
fundosInvestimentoId = '1'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}/control-agencies", headers=headers, params= body)
RenderJSON(r.json())

### Aplicacoes-poupanca

In [33]:
agencia = "551"
contaCorrente = "5420"
variacaoPoupanca = '013'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}", headers=headers, params=body)
RenderJSON(r.json())

In [34]:
agencia = "551"
contaCorrente = "5420"
variacaoPoupanca = '013'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}/orgao-controle", headers=headers, params=body)
RenderJSON(r.json())

## Saldos

### Conta Corrente

In [35]:
agencia = "551"
contaCorrente = "5420"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/saldos/{agencia}-{contaCorrente}/conta-corrente", headers=headers)
RenderJSON(r.json())

### Aplicações financeiras

In [36]:
agencia = "551"
contaCorrente = "5420"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/saldos/{agencia}-{contaCorrente}/aplicacoes-financeiras", headers=headers)
RenderJSON(r.json())

## Orgaos

### Débito

In [45]:
agencia = "551"
contaCorrente = "5420"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'numeroPagina': 1
}

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-debito", headers=headers, params=body)
r.json()

{'erros': [{'codigo': '3164477',
   'versao': '1',
   'mensagem': 'Agência e Conta não está vinculado ao Órgão Repassador.',
   'ocorrencia': 'DCgpo2FaAq1DdQkFiAAA0101'}]}

In [44]:
agencia = "551"
contaCorrente = "5420"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"
ordemBancaria = "1"
item = "1"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-debito/{ordemBancaria}-{item}/categorias-despesa", headers=headers)
r.json()

{'errors': [{'codigo': 404,
   'versao': 1,
   'mensagem': 'Not Found',
   'ocorrencia': 'N/A'}]}

### Crébito

In [43]:
agencia = "551"
contaCorrente = "5420"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'numeroPagina': 1
}

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-credito", headers=headers, params=body)
r.json()


{'statusCode': 404, 'error': 'Not Found', 'message': 'Api not found'}